In [1]:
import pandas as pd
import numpy as np 
import matplotlib 
import time
import pymysql

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

## SQL参数

In [2]:
jdt_db = pymysql.connect(host='172.16.1.231',user='jdys_select',password ='jdys_select2020' ,db= 'jdt_db',charset = 'utf8')

## 导入数据  jdt_user数据

In [3]:
start_time  = time.time()


sql_2022_01 = "SELECT id AS 'id', nickname AS '昵称', gender AS '性别', `level` AS '等级', birthday AS '生日', age AS '年龄', interest_id AS '兴趣ID', work_id AS '行业ID', direct_invite_num AS '直邀用户数', is_sponsor AS '是否主办方', is_lecturer AS '是否讲师', score AS '积分', is_tourist AS '是否游客'  ,is_tourist AS '是否游客', successions AS '连续登录天数', maxsuccessions AS '最大连续登录天数',FROM_UNIXTIME( prevtime, '%Y-%m-%d %H:%i:%s' )  AS '上次登录时间', FROM_UNIXTIME( logintime, '%Y-%m-%d %H:%i:%s' ) AS '登陆时间', FROM_UNIXTIME( updatetime, '%Y-%m-%d %H:%i:%s' ) AS '更新时间', user_province AS '省', user_city AS '市', user_area AS '区', study_time '累积学习时间/秒', last_study_date AS '最后一次学习时间', study_days AS '连续学习天数', total_lesson AS '学习课程数'    ,FROM_UNIXTIME( jointime, '%Y-%m-%d %H:%i:%s' ) AS '加入时间' FROM jdt_user  WHERE    jointime >=  1612108800 AND   jointime < 1614528000"

user_data_2021_01 = pd.read_sql(sql_2022_01,jdt_db)


elapsed = (time.time() - start_time)
print("时间:",elapsed)

时间: 9.68916130065918


## 导入jdt_conference_order用户数据

In [4]:
start_time  = time.time()

sql_conference_order = "SELECT a.buyer_id AS 'id', a.buyer_name AS '姓名', a.buyer_mobile AS '电话',a.is_offline AS '线上/线下会务',COUNT(a.buyer_id) AS '会务订单', a.order_from AS '终端类型'  FROM  jdt_conference_order  AS a  LEFT JOIN jdt_user AS b  ON a.buyer_id = b.id WHERE  b.jointime >= 1612108800  AND    b.jointime <1614528000  GROUP BY  a.is_offline, a.buyer_id"


conference_order = pd.read_sql(sql_conference_order,jdt_db)



elapsed = (time.time() - start_time)
print("时间:",elapsed)

# user_data_2021_three = pd.read_csv(r'D:\DTy_work\dataframe\User_portrait_data\sql_three.csv',sep='\t')

conference_order.columns = ['id','姓名','电话','线上/线下会务','会务订单总数','终端类型']

user_data_2021_three_x = conference_order[conference_order['线上/线下会务'] == 2]
user_data_2021_three_y = conference_order[conference_order['线上/线下会务'] == 1]

user_data_2021_three_x['线上会务订单数'] = user_data_2021_three_x['会务订单总数']
user_data_2021_three_x['线下会务订单数'] =0

user_data_2021_three_y['线下会务订单数'] = user_data_2021_three_y['会务订单总数']
user_data_2021_three_y['线上会务订单数'] = 0

# 会务1,2组合
user_data_2021_three_drop = pd.concat([user_data_2021_three_x,user_data_2021_three_y],axis=0)[['id','姓名','电话','线上/线下会务','会务订单总数','线上会务订单数','线下会务订单数']]

# 1,2组合数据完全干净
user_data_2021_three_clean = user_data_2021_three_drop.drop_duplicates(subset=['id'],keep=False,inplace=False)

# 线上线下会务匹配进行找出  条件3
user_data_2021_three_3 = pd.merge(user_data_2021_three_x,user_data_2021_three_y,on=['id'])

user_data_2021_three_3 = user_data_2021_three_3.rename(columns={'姓名_x':'姓名','电话_x':'电话','线上/线下会务_x':'线上/线下会务','线上会务订单数_x':'线上会务订单数','线下会务订单数_y':'线下会务订单数','会务订单总数_x':'会务订单总数'})[['id','姓名','电话','线上/线下会务','会务订单总数','线上会务订单数','线下会务订单数']]

user_data_2021_three_3['线上/线下会务'] = 3

conference_order_sum = pd.concat([user_data_2021_three_3,user_data_2021_three_clean])

conference_order_sum['会务订单总数'] =conference_order_sum['线上会务订单数'] +  conference_order_sum['线下会务订单数']

user_data_2021_01_sum1 = user_data_2021_01.merge(conference_order_sum,on= 'id',how =  'left')

时间: 1.2868263721466064


C:\Users\cy176\AppData\Local\Temp/ipykernel_9676/130723380.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_data_2021_three_x['线上会务订单数'] = user_data_2021_three_x['会务订单总数']
C:\Users\cy176\AppData\Local\Temp/ipykernel_9676/130723380.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_data_2021_three_x['线下会务订单数'] =0
C:\Users\cy176\AppData\Local\Temp/ipykernel_9676/130723380.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

## 导入jdt_user_bank

In [5]:
start_time  = time.time()


sql_user_bank = "SELECT	user_id as 'id', id_num as '身份证号' from  jdt_user_bank"

user_bank = pd.read_sql(sql_user_bank,jdt_db)


elapsed = (time.time() - start_time)
print("时间:",elapsed)

user_data_2021_01_sum2 = user_data_2021_01_sum1.merge(user_bank,on= 'id',how =  'left')

user_data_2021_01_sum2.head(1)

时间: 2.0026447772979736


,id,昵称,性别,等级,生日,年龄,兴趣ID,行业ID,直邀用户数,是否主办方,是否讲师,积分,是否游客,是否游客,连续登录天数,最大连续登录天数,上次登录时间,登陆时间,更新时间,省,市,区,累积学习时间/秒,最后一次学习时间,连续学习天数,学习课程数,加入时间,姓名,电话,线上/线下会务,会务订单总数,线上会务订单数,线下会务订单数,身份证号
0,850007,,0,1,None,0,None,None,0,0,0,0,0,0,1,1,2021-02-01 00:03:25,2021-02-01 00:03:54,2021-02-01 00:03:54,湖南省,长沙市,None,0,,0,0,2021-02-01 00:03:25,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##  数据清洗，tableau备用

In [6]:
# 数据清洗

In [7]:
user_data_2021_01_sum2['姓名'] = user_data_2021_01_sum2['姓名'].fillna('未知')
user_data_2021_01_sum2['电话'] = user_data_2021_01_sum2['电话'].fillna(0)
user_data_2021_01_sum2['线上/线下会务'] = user_data_2021_01_sum2['线上/线下会务'].fillna(0)
user_data_2021_01_sum2['最后一次学习时间'] = user_data_2021_01_sum2['最后一次学习时间'].fillna(0)
user_data_2021_01_sum2['行业ID'] = user_data_2021_01_sum2['行业ID'].fillna(0)
user_data_2021_01_sum2['兴趣ID'] = user_data_2021_01_sum2['兴趣ID'].fillna(0)
user_data_2021_01_sum2['生日'] = user_data_2021_01_sum2['生日'].fillna(0)
user_data_2021_01_sum2['身份证号'] = user_data_2021_01_sum2['身份证号'].fillna(0)


user_data_2021_01_sum2['new_time'] = pd.datetime.now()

user_data_2021_01_sum2['加入时间'] = pd.to_datetime(user_data_2021_01_sum2['加入时间'], format='%Y-%m-%d')

user_data_2021_01_sum2['register_time'] = (user_data_2021_01_sum2['new_time'] - user_data_2021_01_sum2['加入时间']).apply(lambda x : x.days)


user_data_2021_01_sum2['born'] =  user_data_2021_01_sum2['身份证号'].str[6:10]
user_data_2021_01_sum2['born'] = user_data_2021_01_sum2['born'].fillna(2022)
user_data_2021_01_sum2['born']  = user_data_2021_01_sum2['born'] .astype(int)

user_data_2021_01_sum2['年龄'] = 2022 - user_data_2021_01_sum2['born']

user_data_2021_01_sum2['count'] = 1

C:\Users\cy176\AppData\Local\Temp/ipykernel_9676/615214135.py:11: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  user_data_2021_01_sum2['new_time'] = pd.datetime.now()


In [8]:
user_data_2021_01_sum2.to_csv(r'D:\DTy_work\dataframe\User_portrait_data\to_data\data\user_2021_02.csv')